In [365]:
import random
import math
import agentpy as ap
import numpy as np
# Visualization
import seaborn as sns
import pandas as pd

In [407]:
class RandomAgent(ap.Agent):
    """ Se mueve a celdas al azar """
    
    def setup(self):
        self.position = (0,0)  # Posición actual del agente (x, y)
        self.intention = None  # Intención actual (clean, move o wait)
        self.new_position = None  # Objetivo al moverse (x, y)
        self.is_dirty = False
        
    def see(self):
        """ Percibe su entorno """
        x, y = self.position
        self.is_dirty = self.model.is_dirty[x][y]
        self.neighbors = self.model.get_neighbors(x, y) 
    
    def next(self):
        """ Decide la próxima acción según lo que percibe """
        self.intention = "wait" # por defecto espera
        if self.is_dirty:
            self.intention = "clean"
        else:
            self.new_position = random.choice(self.neighbors)
            if self.model.is_valid_position(self.new_position):
                self.intention = "move"
                
    def action(self):
        """ Ejecuta la accion """
        x, y = self.position
        if self.intention == "clean":
            self.model.clean_cell(x, y)
        elif self.intention == "move":
            self.position = self.new_position
            
    def move(self):
        if not self.model.is_board_clean:
            self.model.total_moves += 1     
        self.see()
        self.next()
        self.action()

        

In [401]:
class SergioAgent(ap.Agent):
    """ Empieza con una estrategia de llenado en columnas, seguido de busqueda por barrido """
    
    def setup(self):
        self.position = (0,0)  # Posición actual del agente (x, y)
        self.intention = None  # Intención actual (clean, move o wait)
        self.new_position = None  # Objetivo al moverse (x, y)
        self.is_dirty = False
        
    def see(self):
        """ Percibe su entorno """
        x, y = self.position
        self.is_dirty = self.model.is_dirty[x][y]
        self.neighbors = self.model.get_neighbors(x, y) 
    
    def next(self):
        """ Decide la próxima acción según lo que percibe """
        self.intention = "wait" # por defecto espera
        if self.is_dirty:
            self.intention = "clean"
        else:
            self.new_position = random.choice(self.neighbors)
            if self.model.is_valid_position(self.new_position):
                self.intention = "move"
                
    def action(self):
        """ Ejecuta la accion """
        x, y = self.position
        if self.intention == "clean":
            self.model.clean_cell(x, y)
        elif self.intention == "move":
            self.position = self.new_position
            
    def move(self):
        if not self.model.is_board_clean:
            self.model.total_moves += 1     
        self.see()
        self.next()
        self.action()


In [402]:
class RodrigoAgent(ap.Agent):
    
    def setup(self):
        pass
        
    def see(self):
        pass 
    
    def next(self):
        pass
                
    def action(self):
       pass
            
    def move(self):
        if not self.model.is_board_clean:
            self.model.total_moves += 1     
        self.see()
        self.next()
        self.action()


In [403]:
class OscarAgent(ap.Agent):
    
    def setup(self):
        pass
        
    def see(self):
        pass 
    
    def next(self):
        pass
                
    def action(self):
       pass
            
    def move(self):
        if not self.model.is_board_clean:
            self.model.total_moves += 1     
        self.see()
        self.next()
        self.action()


In [404]:
class PepeAgent(ap.Agent):
    
    def setup(self):
        pass
        
    def see(self):
        pass 
    
    def next(self):
        pass
                
    def action(self):
       pass
            
    def move(self):
        if not self.model.is_board_clean:
            self.model.total_moves += 1     
        self.see()
        self.next()
        self.action()


In [405]:
class HectorAgent(ap.Agent):
    
    def setup(self):
        pass
        
    def see(self):
        pass 
    
    def next(self):
        pass
                
    def action(self):
       pass
            
    def move(self):
        if not self.model.is_board_clean:
            self.model.total_moves += 1     
        self.see()
        self.next()
        self.action()


In [406]:
class CleaningModel(ap.Model):
    """ Modelo de limpieza de tablero """
    
    def generateDirtyCells(self):
        """
        Genera una matriz n x m donde un porcentaje de las celdas están marcadas como sucias (1) y el resto limpias (0).
        devuelve: Matriz n x m con celdas sucias y limpias
        """
        totalCells = self.p.n * self.p.m
    
        dirtyCellsCount = math.ceil(totalCells * (self.p.percentage_dirty / 100))

        # Inicializamos la matriz con ceros
        isDirty = [[0 for _ in range(self.p.m)] for _ in range(self.p.n)]

        # Generamos las posiciones de las celdas sucias de manera aleatoria
        allPositions = [(i, j) for i in range(self.p.n) for j in range(self.p.m)]
        dirtyPositions = random.sample(allPositions, dirtyCellsCount)

        # Marcamos las celdas seleccionadas como sucias
        for i, j in dirtyPositions:
            isDirty[i][j] = 1

        return isDirty
    
    def is_valid_position(self, position):
        """ Verifica si una posición es valida dentro de nuestro tablero """
        x, y = position
        return 0 <= x < self.p.n and 0 <= y < self.p.m
    
    def get_neighbors(self, x, y):
        """ regresa lista con todas las celdas vecinas de la posicion x, y"""
        neighbors = []
        
        for dx in range(-1, 2):  # Iterata -1, 0, 1
            for dy in range(-1, 2):  # Itera -1, 0, 1
                if not (dx == 0 and dy == 0):  # Saltamos la celda actual
                    neighbors.append((x + dx, y + dy)) 
    
        return neighbors
    
    def clean_cell(self, x, y): 
        self.is_dirty[x][y] = 0 
        self.cleaned_cells += 1
        if not(self.is_board_clean) and self.cleaned_cells == self.dirty_cells:
            self.finish_time = self.t
            self.is_board_clean = True

    
    def setup(self):
        
        self.random_agents = ap.AgentList(self, self.p.random_agents, RandomAgent)
        self.sergio_agents = ap.AgentList(self, self.p.sergio_agents, SergioAgent)
        self.rodrigo_agents = ap.AgentList(self, self.p.rodrigo_agents, RodrigoAgent)
        self.oscar_agents = ap.AgentList(self, self.p.oscar_agents, OscarAgent)
        self.pepe_agents = ap.AgentList(self, self.p.pepe_agents, PepeAgent)
        self.hector_agents = ap.AgentList(self, self.p.hector_agents, HectorAgent)
        

        # self.all_agents = self.sergio_agents + self.rodrigo_agents + self.oscar_agents + self.pepe_agents + self.hector_agents
        self.all_agents = self.random_agents + self.sergio_agents + self.rodrigo_agents + self.oscar_agents + self.pepe_agents + self.hector_agents
        # Genera matriz donde isDirty[i][j] == 1 si la celda en la fila i y columna j esta sucia
        self.is_dirty = self.generateDirtyCells()
        
        # Inicializamos la matriz con ceros
        agentCountMatrix = [[0 for _ in range(self.p.m)] for _ in range(self.p.n)]
        agentCountMatrix[0][0] = len(self.all_agents)
        
        # Variables para generar estadisticas
        self.total_moves = 0
        self.cleaned_cells = 0
        self.dirty_cells = math.ceil(self.p.n * self.p.m * (self.p.percentage_dirty / 100))
        self.is_board_clean = True if self.p.percentage_dirty == 0 else False # Es falso a menos que no haya celdas sucias
        self.finish_time = self.p.steps + 1 # Por default no termina en tiempo


    def step(self):
        self.all_agents.move()

    def update(self):
        """
        self.record('Gini Coefficient (all_agents)', gini(self.all_agents.wealth))
        self.record('Gini Coefficient (sergio_agents)', gini(self.sergio_agents.wealth))
        self.record('Gini Coefficient (rodrigo_agents)', gini(self.rodrigo_agents.wealth))
        self.record('Gini Coefficient (oscar_agents)', gini(self.oscar_agents.wealth))
        self.record('Gini Coefficient (pepe_agents)', gini(self.pepe_agents.wealth))
        self.record('Gini Coefficient (hector_agents)', gini(self.hector_agents.wealth))

        self.record('State Utility (all_agents)', self.all_agents.state_utility)
        self.record('State Utility (sergio_agents)', self.sergio_agents.state_utility)
        self.record('State Utility (rodrigo_agents)', self.rodrigo_agents.state_utility)
        self.record('State Utility (oscar_agents)', self.oscar_agents.state_utility)
        self.record('State Utility (pepe_agents)', self.pepe_agents.state_utility)
        self.record('State Utility (hector_agents)', self.hector_agents.state_utility)

        self.record('Run Utility (all_agents)', self.all_agents.run_utility)
        self.record('Run Utility (sergio_agents)', self.sergio_agents.run_utility)
        self.record('Run Utility (rodrigo_agents)', self.rodrigo_agents.run_utility)
        self.record('Run Utility (oscar_agents)', self.oscar_agents.run_utility)
        self.record('Run Utility (pepe_agents)', self.pepe_agents.run_utility)
        self.record('Run Utility (hector_agents)', self.hector_agents.run_utility)

        self.record('Predicate (all_agents)', self.all_agents.predicate)
        self.record('Predicate (sergio_agents)', self.sergio_agents.predicate)
        self.record('Predicate (rodrigo_agents)', self.rodrigo_agents.predicate)
        self.record('Predicate (oscar_agents)', self.oscar_agents.predicate)
        self.record('Predicate (pepe_agents)', self.pepe_agents.predicate)
        self.record('Predicate (hector_agents)', self.hector_agents.predicate)
    """

    def end(self):
        time_taken = self.finish_time
        cleaned_percentage = (self.cleaned_cells / (self.dirty_cells)) * 100
        clean_percentage = ((self.p.n * self.p.m - self.cleaned_cells) / (self.p.n * self.p.m)) * 100
        print()
        
        if not self.is_board_clean:
            print("La limpieza no acabo, se llego al tiempo maximo")
        else:
            print(f"Tiempo de limpieza: {time_taken} pasos")
            
        print(f"Porcentaje de celdas limpiadas: {cleaned_percentage:.2f}%")
        print(f"Porcentaje de celdas limpias: {clean_percentage:.2f}%")
        print(f"Total de movimientos: {self.total_moves}")
        
        """
        self.all_agents.record('wealth')
        self.all_agents.record('run_utility')
        self.all_agents.record('state_utility')
        self.all_agents.record('predicate')
        """

In [408]:
parameters = {
    'random_agents': 5,
    'sergio_agents': 5,
    'rodrigo_agents': 5,
    'oscar_agents': 5,
    'pepe_agents': 5,
    'hector_agents': 5,
    'steps': 30,
    'n': 5,
    'm': 5,
    'percentage_dirty': 15,
    'seed': 22,
}

In [437]:
model = CleaningModel(parameters)
results = model.run()

Completed: 30 steps
Tiempo de limpieza: 23 pasos
Porcentaje de celdas limpiadas: 100.00%
Porcentaje de celdas limpias: 84.00%
Total de movimientos: 662

Run time: 0:00:00.002524
Simulation finished
